In [ ]:
# Carga básica y segura para Google Colab
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)


In [ ]:
# Importar solo las librerías necesarias sin causar conflictos
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mostrar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)



In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cargar los datos
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Convertir etiquetas a binario
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values


In [ ]:
# Importar librerías necesarias
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D

# Parámetros
vocab_size = 10000
max_len = 100
embedding_dim = 16

# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_messages)

# Convertir textos a secuencias
train_seq = tokenizer.texts_to_sequences(train_messages)
train_pad = pad_sequences(train_seq, maxlen=max_len, padding='post')

test_seq = tokenizer.texts_to_sequences(test_messages)
test_pad = pad_sequences(test_seq, maxlen=max_len, padding='post')

# Modelo
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento
model.fit(train_pad, train_labels, epochs=10, validation_data=(test_pad, test_labels))

# Función para predecir mensajes
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    pad = pad_sequences(seq, maxlen=max_len, padding='post')
    prob = float(model.predict(pad)[0][0])
    label = "spam" if prob > 0.5 else "ham"
    return [prob, label]

# Probar la función con un mensaje
pred_text = "how are you doing today?"
print(predict_message(pred_text))


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
